In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32 #100
BATCH_SIZE = 100 #32
EPOCHS = 500

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("adversarial_images")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(LATENT_DIM)(x)
z_log_var = layers.Dense(LATENT_DIM)(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(32*32*64, activation='relu')(decoder_inputs)
x = layers.Reshape((32, 32, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("adversarial_cifar10_images.npy", embeddings)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/500
5000/5000 [==============================] - 52s 10ms/sample - loss: 33188.8615
Epoch 2/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 31709.0073
Epoch 3/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 30346.3429
Epoch 4/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 29549.8561
Epoch 5/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 29283.1712
Epoch 6/500
5000/5000 [==============================] - 50s 10ms/sample - loss: 29116.6366
Epoch 7/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 29036.9740
Epoch 8/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 29004.9786
Epoch 9/500
5000/5000 [=============================

5000/5000 [==============================] - 51s 10ms/sample - loss: 27929.3966
Epoch 82/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27929.6075
Epoch 83/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27945.5939
Epoch 84/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27956.2790
Epoch 85/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27924.0905
Epoch 86/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27914.8939
Epoch 87/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27905.5502
Epoch 88/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27904.8636
Epoch 89/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27913.0858
Epoch 90/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27919.0500
Epoch 91/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 2

5000/5000 [==============================] - 51s 10ms/sample - loss: 27785.3261
Epoch 169/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27788.4154
Epoch 170/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27781.8269
Epoch 171/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27785.0221
Epoch 172/500
5000/5000 [==============================] - 52s 10ms/sample - loss: 27800.6043
Epoch 173/500
5000/5000 [==============================] - 52s 10ms/sample - loss: 27779.7236
Epoch 174/500
5000/5000 [==============================] - 52s 10ms/sample - loss: 27772.4756
Epoch 175/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27766.8752
Epoch 176/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27770.4478
Epoch 177/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27775.5343
Epoch 178/500
5000/5000 [==============================] - 51s 10ms/sample

5000/5000 [==============================] - 51s 10ms/sample - loss: 27720.9163
Epoch 256/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27723.8220
Epoch 257/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27730.6298
Epoch 258/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27745.0905
Epoch 259/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27732.9270
Epoch 260/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27722.7137
Epoch 261/500
5000/5000 [==============================] - 50s 10ms/sample - loss: 27717.7993
Epoch 262/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27717.3592
Epoch 263/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27715.9518
Epoch 264/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27717.3219
Epoch 265/500
5000/5000 [==============================] - 51s 10ms/sample

5000/5000 [==============================] - 51s 10ms/sample - loss: 27693.3058
Epoch 343/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27700.6259
Epoch 344/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27690.4181
Epoch 345/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27690.6245
Epoch 346/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27698.6912
Epoch 347/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27690.2015
Epoch 348/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27685.8051
Epoch 349/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27685.3221
Epoch 350/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27685.9415
Epoch 351/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27688.2004
Epoch 352/500
5000/5000 [==============================] - 51s 10ms/sample

5000/5000 [==============================] - 51s 10ms/sample - loss: 27667.0869
Epoch 430/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27672.7288
Epoch 431/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27668.9248
Epoch 432/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27666.7286
Epoch 433/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27668.6049
Epoch 434/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27670.0911
Epoch 435/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27670.8909
Epoch 436/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27668.0161
Epoch 437/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27663.8510
Epoch 438/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 27662.7359
Epoch 439/500
5000/5000 [==============================] - 51s 10ms/sample

In [2]:
import numpy as np

# Load the .npy file
data = np.load('adversarial_cifar10_images.npy')

# Check the shape
print("Shape of the array:", data.shape)


('Shape of the array:', (5000, 32))


In [3]:
data

array([[ 1.5836649e+00, -7.4605726e-02, -1.8275051e-01, ...,
         6.6403650e-02,  4.5942271e-01,  2.4671441e-01],
       [-1.6811882e-01, -2.1509863e-01, -1.2463778e-03, ...,
         1.7918052e-01,  1.0419154e-01,  8.3651674e-01],
       [ 1.7436147e+00,  1.1344336e+00, -1.1550640e+00, ...,
        -2.4377705e-01, -1.2210477e-02,  7.4439156e-01],
       ...,
       [ 1.7111864e+00, -1.9551393e+00,  1.8093244e+00, ...,
         1.1281130e+00, -3.6323097e-01, -5.9465039e-01],
       [-1.7370853e-01,  1.5486891e+00,  1.2929791e+00, ...,
        -3.0171707e-01, -1.4915882e+00, -9.7394443e-01],
       [ 4.6484253e-01,  1.5378593e+00,  1.5952064e-01, ...,
         8.0008340e-01,  1.5868562e+00, -9.8196292e-01]], dtype=float32)

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32 #100
BATCH_SIZE = 100 #32
EPOCHS = 500

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("cifar10_original")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(LATENT_DIM)(x)
z_log_var = layers.Dense(LATENT_DIM)(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(32*32*64, activation='relu')(decoder_inputs)
x = layers.Reshape((32, 32, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("cifar10_original.npy", embeddings)

Epoch 1/500
5000/5000 [==============================] - 53s 11ms/sample - loss: 27731.1131
Epoch 2/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 22751.2107
Epoch 3/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 21214.2430
Epoch 4/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 20336.2301
Epoch 5/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 19719.7327
Epoch 6/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 18848.5729
Epoch 7/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 15510.4954
Epoch 8/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 14314.3617
Epoch 9/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 13916.7472
Epoch 10/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 13679.5311
Epoch 11/500
5000/5000 [==============================] - 51s 10ms/sample - los

5000/5000 [==============================] - 51s 10ms/sample - loss: 8659.5853
Epoch 90/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8675.2782
Epoch 91/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8731.2590
Epoch 92/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8762.6141
Epoch 93/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8681.0739
Epoch 94/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8576.3842
Epoch 95/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8522.9821
Epoch 96/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8556.0597
Epoch 97/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8616.1053
Epoch 98/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8586.1426
Epoch 99/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8533.6562
E

5000/5000 [==============================] - 51s 10ms/sample - loss: 8055.7917
Epoch 178/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8049.6767
Epoch 179/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8026.0195
Epoch 180/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8002.4890
Epoch 181/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7992.8311
Epoch 182/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8018.7749
Epoch 183/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8051.6277
Epoch 184/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8031.3960
Epoch 185/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8017.5718
Epoch 186/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8008.6474
Epoch 187/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 8

5000/5000 [==============================] - 51s 10ms/sample - loss: 7787.4714
Epoch 266/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7848.3318
Epoch 267/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7865.4547
Epoch 268/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7837.0644
Epoch 269/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7825.5283
Epoch 270/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7853.4458
Epoch 271/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7806.3314
Epoch 272/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7758.8150
Epoch 273/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7738.7174
Epoch 274/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7741.8704
Epoch 275/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7

5000/5000 [==============================] - 51s 10ms/sample - loss: 7618.8351
Epoch 354/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7604.9394
Epoch 355/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7605.3822
Epoch 356/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7600.0027
Epoch 357/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7613.9581
Epoch 358/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7634.1067
Epoch 359/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7648.2183
Epoch 360/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7664.3219
Epoch 361/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7668.3788
Epoch 362/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7664.4432
Epoch 363/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7

5000/5000 [==============================] - 51s 10ms/sample - loss: 7460.0149
Epoch 442/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7481.7370
Epoch 443/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7505.0005
Epoch 444/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7524.8183
Epoch 445/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7534.7028
Epoch 446/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7539.2238
Epoch 447/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7537.1555
Epoch 448/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7524.0153
Epoch 449/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7513.1615
Epoch 450/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7518.5872
Epoch 451/500
5000/5000 [==============================] - 51s 10ms/sample - loss: 7